In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import subprocess
from sklearn.metrics import r2_score, confusion_matrix, classification_report, mean_squared_error
from scipy.stats import spearmanr
%matplotlib inline

In [7]:
nvals = [128, 256]
cnns = ["1d", "2d"]
blocks = range(1, 4)
scenarios = [f"demo_{e}_n{n}_cosine_{cnn}_{b}blocks" for n in nvals for cnn in cnns for b in blocks for e in ["01", "0255", "neg11"]]
names = ["N0", "t1", "N1", "t2", "N2"]

In [ ]:
res_dict = {i:{} for i in scenarios for n in names}
for s in scenarios:
        preds = pd.read_csv(f"demo/{s}/{s}_preds.csv")
        plot, axes = plt.subplots(1, len(names))
        plot.set_size_inches(25, 5)

        for idx, name in enumerate(names):
                axes[idx].scatter(preds[f"true_{name}"], preds[f"pred_{name}"])
                m, b = np.polyfit(preds[f"true_{name}"], preds[f"pred_{name}"], 1)
                mse = mean_squared_error(preds[f"true_{name}"], preds[f"pred_{name}"], squared=False)
                res_dict[s][name] = mse
                linevals = np.linspace(np.min(preds[f"true_{name}"]), np.max(preds[f"true_{name}"]))
                axes[idx].plot(linevals, linevals, color="black", label=f'''RMSE: {mse:.2f}\nSpearmans rho: {spearmanr(preds[f"true_{name}"], preds[f"pred_{name}"])[0]:.2f}''')
                        
                axes[idx].legend()
                axes[idx].set_title(name)
                axes[idx].set_ylabel(f"True log({name})")
                axes[idx].set_xlabel(f"Pred log({name})")
                
        plt.suptitle(s)
        plt.tight_layout()
        plt.savefig(f"demo/{s}_mergedfig.png")
        


In [16]:
df =  pd.DataFrame(res_dict).transpose()
df = df.rename_axis("scenarios").reset_index()
df["mean"] = df[names].mean(axis=1)
print(df)
df.to_csv("demo/pscaled_all_mses.csv", index=False)

                            scenarios        N0        t1        N1        t2   
0      demo_01_n128_cosine_1d_1blocks  0.724319  1.082283  0.689953  0.869464  \
1    demo_0255_n128_cosine_1d_1blocks  0.723246  1.072375  0.732597  0.864746   
2   demo_neg11_n128_cosine_1d_1blocks  0.677074  1.046497  0.712347  0.825973   
3      demo_01_n128_cosine_1d_2blocks  0.697282  1.061628  0.694153  0.842301   
4    demo_0255_n128_cosine_1d_2blocks  0.748351  1.094881  0.715027  0.891323   
5   demo_neg11_n128_cosine_1d_2blocks  0.695632  1.059587  0.717682  0.843032   
6      demo_01_n128_cosine_1d_3blocks  0.682375  1.051477  0.687455  0.832270   
7    demo_0255_n128_cosine_1d_3blocks  0.722740  1.061793  0.709566  0.860464   
8   demo_neg11_n128_cosine_1d_3blocks  0.684338  1.059699  0.709794  0.838752   
9      demo_01_n128_cosine_2d_1blocks  0.554039  0.906030  0.674732  0.717482   
10   demo_0255_n128_cosine_2d_1blocks  0.613328  1.007008  0.695791  0.782608   
11  demo_neg11_n128_cosine_2

In [ ]:
df

,scenarios,N0,t1,N1,t2,N2,mean
0,n128_cosine_1dcnn_log_1blocks,0.496614,1.129429,0.478558,0.728475,0.530143,0.672644
1,n128_cosine_1dcnn_log_2blocks,0.505948,1.176965,0.484331,0.743520,0.546777,0.691508
2,n128_cosine_1dcnn_log_3blocks,0.487770,1.124151,0.490473,0.724229,0.503112,0.665947
3,n128_cosine_2dcnn_log_1blocks,0.387732,0.884812,0.435924,0.586924,0.331561,0.525391
4,n128_cosine_2dcnn_log_2blocks,0.374800,0.849150,0.438229,0.567272,0.331822,0.512255
5,n128_cosine_2dcnn_log_3blocks,0.381448,0.894858,0.443760,0.589872,0.328817,0.527751
6,n256_cosine_1dcnn_log_1blocks,0.482819,1.178084,0.503166,0.732207,0.415043,0.662264
7,n256_cosine_1dcnn_log_2blocks,0.495432,1.176717,0.497549,0.752909,0.401679,0.664857
8,n256_cosine_1dcnn_log_3blocks,0.462545,1.126429,0.475594,0.710261,0.382637,0.631493
9,n256_cosine_2dcnn_log_1blocks,0.307276,0.838180,0.401692,0.523419,0.298443,0.473802


In [ ]:
subprocess.run(f"convert -append {' '.join([f'demo/results/{s}_mergedfig.png' for s in scenarios])} demo_res.png", shell=True)

CompletedProcess(args='convert -append demo/results/n128_cosine_1dcnn_log_1blocks_mergedfig.png demo/results/n128_cosine_1dcnn_log_2blocks_mergedfig.png demo/results/n128_cosine_1dcnn_log_3blocks_mergedfig.png demo/results/n128_cosine_2dcnn_log_1blocks_mergedfig.png demo/results/n128_cosine_2dcnn_log_2blocks_mergedfig.png demo/results/n128_cosine_2dcnn_log_3blocks_mergedfig.png demo/results/n256_cosine_1dcnn_log_1blocks_mergedfig.png demo/results/n256_cosine_1dcnn_log_2blocks_mergedfig.png demo/results/n256_cosine_1dcnn_log_3blocks_mergedfig.png demo/results/n256_cosine_2dcnn_log_1blocks_mergedfig.png demo/results/n256_cosine_2dcnn_log_2blocks_mergedfig.png demo/results/n256_cosine_2dcnn_log_3blocks_mergedfig.png demo_res.png', returncode=0)

In [ ]:
intro_df = pd.read_csv("1dcnn_results.csv", sep="\t")
intro_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '1dcnn_results.csv'

In [ ]:
print(classification_report(intro_df["true"], intro_df["pred"]))

              precision    recall  f1-score   support

          ab       0.93      0.76      0.84     42880
          ba       1.00      0.97      0.98     42880
        none       0.79      0.96      0.87     42880

    accuracy                           0.90    128640
   macro avg       0.90      0.90      0.89    128640
weighted avg       0.90      0.90      0.89    128640

